In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122597")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122597
Azure region: southcentralus
Subscription id: bc4ac49b-f3f0-45bb-995b-638f2c80696f
Resource group: aml-quickstarts-122597


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "BankCC "

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import Dataset
import os

# Specify parameter sampler
ps = RandomParameterSampling(
	{
		'--C':uniform(.90, 1.10),
		'--max_iter':choice(80, 90, 100, 110, 120)
	}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target='BankCC' , entry_script='train.py', pip_packages=[])

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=4)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
import joblib
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Best Accuracy from runs:", best_run_metrics["Accuracy"]  )

model = best_run.register_model(model_name = "HyperdriveBest", model_path = "outputs/model.joblib")


Best Accuracy from runs: 0.910980320029428


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory


# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
file_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dsauto = TabularDatasetFactory.from_delimited_files(path = file_url)

In [7]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(dsauto)
y_df = pd.DataFrame(y, columns = ['y'])
df = pd.concat([x, y_df], axis = 1)
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size = .33, random_state = 7)
train_data.to_csv("training/training_data.csv")
experiment = Experiment(ws, "automl_experiment")
datastore = ws.get_default_datastore()
datastore.upload(src_dir = "training/", target_path = "data/")
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_data.csv"))])

Uploading an estimated of 1 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [9]:
from azureml.train.automl import AutoMLConfig



# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [24]:
# Submit your automl run
automlrun = experiment.submit(config = automl_config, show_output = True)
RunDetails(automlrun).show()
automlrun.wait_for_completion()

Running on local machine
Parent Run ID: AutoML_2f61e8a9-0c50-4dc4-b705-bbf802588170

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [31]:
# Retrieve and save your best automl model.
best_automlrun = automlrun.get_output()
print(best_automlrun)

best_automlrun.register_model(model_name = "AutomlBest", path = "./outputs/")

(Run(Experiment: automl_experiment,
Id: AutoML_2f61e8a9-0c50-4dc4-b705-bbf802588170_39,
Type: None,
Status: Completed), Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_alpha=2.291666666666667,
                                                                                               reg_lambda=0.3125,
                                  

In [45]:
cpu_cluster.delete()

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 500
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '1451', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-failure-cause': 'service', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'error', 'x-ms-client-request-id': '8dc15e65-12e5-439b-b37e-b07d19eded8d', 'x-ms-client-session-id': 'dedf7b65-6aa8-49d0-bb29-91a8d56eae41', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.109', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-ratelimit-remaining-subscription-deletes': '14999', 'x-ms-request-id': '8c9b78f0-4967-405e-bcc8-8b39a4c73b51', 'x-ms-correlation-request-id': '8c9b78f0-4967-405e-bcc8-8b39a4c73b51', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20201024T074732Z:8c9b78f0-4967-405e-bcc8-8b39a4c73b51', 'Date': 'Sat, 24 Oct 2020 07:47:31 